In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import pandas as pd

In [ ]:
!which chromedriver

In [75]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [3]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [6]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []


today = datetime.datetime.now()

for product in products:

    # Get the link to the product page.
    link = product.find('a')
    href = link['href']
    product_url = base_url + href
    productURL_list.append(product_url)

    # Get the link to the product image.
    img = product.find('img')['src']
    productImage_list.append(img)

    # Get the product title.
    product_title = product.find('img')['alt']
    productTitle_list.append(product_title)

    # Try to get the number of reviews, there may not be any.
    try:
        review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
        reviewAmount_list.append(review_amount)
        
    except:
        review_amount = 0
        reviewAmount_list.append(review_amount)

    # In the case where there are at least one review...
    if int(review_amount) > 0:
        # Get the average number of stars
        stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
        starReview_list.append(stars_review)
        
        # Get the URL to the reviews section for that product.
        review_url = product.find('div', class_='stars').find('a')['href']
        reviewURL_list.append(base_url+review_url)

    # Otherwise, use defalt 0 or NaN values for these entries.
    else:
        stars_review = 0
        starReview_list.append(stars_review)
        
        review_url = "NaN"
        reviewURL_list.append(review_url)
        
    # Get the price of the product.
    price = product.find('span', class_='price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    currentPrice_list.append(current_price)
    
# Close the browser window
# browser.quit()
    
# Create a dictionary with the lists of the scrapped data.
data = {
    "Title": productTitle_list,
    "URL": productURL_list,
    "Image": productImage_list,
    "AverageStars": starReview_list,
    "NumberofReviews": reviewAmount_list,
    "ReviewsURL": reviewURL_list,
    "Price": currentPrice_list
       }

# Create a Pandas DataFrame with that dictionary
product_df = pd.DataFrame.from_dict(data)
product_df.head()

,Title,URL,Image,AverageStars,NumberofReviews,ReviewsURL,Price
0,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,https://i5.walmartimages.com/asr/ca7e3b65-d3c8...,4.3,517,https://www.walmart.com/ip/Holmes-Personal-Spa...,$14.98
1,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,https://i5.walmartimages.com/asr/5ba9740e-a3f1...,4.4,136,https://www.walmart.com/ip/High-Efficiency-HEP...,$49.69
2,Filtrete F2 Bacteria & Virus True HEPA Room Ai...,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,https://i5.walmartimages.com/asr/dea94bac-be28...,4.8,55,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,$17.97
3,Winix 5500-2 Air Cleaner with PlasmaWave Techn...,https://www.walmart.com/ip/Winix-5500-2-Air-Cl...,https://i5.walmartimages.com/asr/7e1f8414-d0c2...,4.7,89,https://www.walmart.com/ip/Winix-5500-2-Air-Cl...,$159.99
4,Holmes Allergen Removing Air Purifier with Tru...,https://www.walmart.com/ip/Holmes-Allergen-Rem...,https://i5.walmartimages.com/asr/c3cedca5-8b4a...,4.4,182,https://www.walmart.com/ip/Holmes-Allergen-Rem...,$130.00


In [7]:
product_df = product_df.astype({
    "NumberofReviews": 'int',
    "AverageStars": 'float'
})

product_df.dtypes

Title               object
URL                 object
Image               object
AverageStars       float64
NumberofReviews      int64
ReviewsURL          object
Price               object
dtype: object

In [71]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,6):
    if product_df['NumberofReviews'][i] > 0:
        
        productTitleList = []
        reviewURLList = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
        if len(reviewRatingList) < product_df['NumberofReviews'][i]:
#         while len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
            try:
                browser.visit(product_df["ReviewsURL"][i])
                html = browser.html
                soup = BeautifulSoup(html, 'html.parser')
                allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
                browser.visit(base_url+allReviewsURL)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find_all('div', class_='ReviewList-content')

                # need to add a way to make sure I am capturing all of the reviews
                # can check this by looking at NumberofReview for this ReviewsRUL
                # to scroll down a little use the below command
                # browser.execute_script("window.scrollTo(200, document.body.scrollHeight);")
                # check the average rating by using the values found here



                for review in reviews:

                    productTitleList.append(product_df['Title'][i])
                    reviewURLList.append(product_df['ReviewsURL'][i])

                    try:
                        title = review.find('h3', class_='review-title').text
                        reviewTitleList.append(title)
                        print(title)

                    except:
                        title = 'None'
                        reviewTitleList.append(title)
                        print(title)
#                         pass

                    # adding try only because the browser needs to scroll
                    # once this is added, there wont need to be a try
                    # each review has to have a star value
                    try:
                        reviewRating = review.find('span', class_='seo-avg-rating').text
                        reviewRatingList.append(reviewRating)
                        print(reviewRating)
                    except:
                        pass

                    # adding try and except for the review body for same reason as stars
                    try:
                        reviewComment = review.find('p').text
                        reviewCommentList.append(reviewComment)
#                         print(reviewComment)
                    except:
                        pass
                
            
                # only until I fix the scroll
                reviewTitleList.pop()
                productTitleList.pop()
                reviewURLList.pop()
                
                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                productReviews_df = pd.DataFrame.from_dict(data)
                productReviews_df.head()
                print('into dataframe')

            except:
                pass
    
        print('add to existing dataframe')
        review_df = review_df.append(productReviews_df, ignore_index=True)
print('done')
#                 browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

Its wonderful
5.0
Works great!
5.0
Extremely impressed with performance.
5.0
Great for allergies.
5.0
Absolutely Love this
5.0
Works well
4.0
A bit loud
4.0
Good purifier. Beware of home shipping.
4.0
Good product
4.0
Poorly designed. Air goes right around filters!
1.0
None
stored data
into dataframe
add to existing dataframe
Performs Great
5.0
Disappointed this isn't in stores anymore.
5.0
Envion air purifiers
5.0
Get one
5.0
Therapure Air Purifier, TPP230H
5.0
Air Quality
5.0
Air purifier
5.0
A+
5.0
excellent product
5.0
Working well
4.0
None
stored data
into dataframe
add to existing dataframe
done


In [70]:
# len(review_df)
# review_df.tail()
review_df.describe()

,ProductTitle,ReviewURL,ReviewTitle,ReviewStarRating,ReviewComment
count,20,20,20,20,20
unique,1,1,10,2,10
top,"Envion Therapure 230H, UV Germicidal HEPA Styl...",https://www.walmart.com/ip/Envion-Therapure-23...,"Therapure Air Purifier, TPP230H",5.0,It's came fast and works great so fare
freq,20,20,2,18,2


In [ ]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('span', class_='seo-avg-rating').text
# starts = reviews.find('span', class_='seo-average-rating')
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

In [ ]:
reviews = soup.find_all('div', class_='ReviewList-content')

In [80]:
# Get reviews 

review_df = pd.DataFrame()

# for i in range(len(product_df["ReviewsURL"])):
for i in range(4,5):
    if product_df['NumberofReviews'][i] > 0:
#     number_of_reviews = product_df['NumberofReviews'][i]
#     for j in range(number_of_reviews):
        
        productTitleList = []
        reviewURLList = []
        reviewSiteLink = []
        reviewTitleList = []
        reviewRatingList = []
        reviewCommentList = []
        
#         if len(reviewRatingList) < product_df['NumberofReviews'][i]:
        
#         while len(reviewRatingList) < product_df['NumberofReviews'][i]:
    
        try:
            # go to the product url link
            browser.visit(product_df["ReviewsURL"][i])
            html = browser.html
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the 'see all reviews' button and go there
            allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
            browser.visit(base_url+allReviewsURL)
            soup = BeautifulSoup(html, 'html.parser')
            
            # find the list of reviews
            reviews = soup.find_all('div', class_='ReviewList-content')
            
#             for review in reviews:
            for j in range(len(reviews)):
                reviewSiteLink.append(review)
                reviewSiteLink[j].send_keys(Keys.PAGE_DOWN)
                soup = BeautifulSoup(html, 'html.parser')
                reviews = soup.find('div', class_='ReviewList-content')
                
#                 print(f"j={j}")
                productTitleList.append(product_df['Title'][i])
                reviewURLList.append(product_df['ReviewsURL'][i])
                title, stars, comments = scrapeReviews(review)
                reviewTitleList.append(title)
                reviewRatingList.append(stars)
                reviewCommentList.append(comments)


                data = {
                    'ProductTitle': productTitleList,
                    'ReviewURL': reviewURLList,
                    'ReviewTitle': reviewTitleList,
                    'ReviewStarRating': reviewRatingList,
                    'ReviewComment': reviewCommentList
                }
                print('stored data')
                productReviews_df = pd.DataFrame.from_dict(data)
                productReviews_df.head()
                print('into dataframe')

        except:
            pass

        print('add to existing dataframe')
        review_df = review_df.append(productReviews_df, ignore_index=True)
print('done')

add to existing dataframe
done


In [81]:
# len(review_df)
# review_df.tail()
review_df.describe()

,ProductTitle,ReviewURL,ReviewTitle,ReviewStarRating,ReviewComment
count,10,10,10,10,10
unique,1,1,10,2,10
top,"Envion Therapure 230H, UV Germicidal HEPA Styl...",https://www.walmart.com/ip/Envion-Therapure-23...,"Therapure Air Purifier, TPP230H",5.0,It's came fast and works great so fare
freq,10,10,1,9,1


In [72]:
def scrapeReviews(review):
    
    # get the title of the review
    try:
        title = review.find('h3', class_='review-title').text
    except:
        title = 'None'
        
    # get the number of stars in this review
    reviewRating = review.find('span', class_='seo-avg-rating').text
    
    # get the context of the review
    reviewComment = review.find('p').text
        
    # return these items as a tuple
    return title, stars, comments

In [ ]:
browser.quit()

In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")